# Sample for KFServing SDK v1beta1

This is a sample for KFServing SDK v1beta1. 

### Input variables


In [ ]:
namespace = utils.get_default_target_namespace()
inferenceServiceName = "randomforest"
inFerenceServiceStorageUri = "gs://pipelines_artifacts/kfserving/sklearn/randomforest"
inputFile = "https://github.com/ydataai/academy/blob/chore/fix_tutorials/tutorials/Model%20Serving/KfServing/input_files/input.json"


In [ ]:
from kubernetes import client 
from kfserving import *

## Define InferenceService

Define the inferenceservice basic on the endpoint spec.

In [ ]:
api_version = constants.KFSERVING_GROUP + "/v1beta1"

annotations = {}
annotations["sidecar.istio.io/inject"] = "false"
isvc = V1beta1InferenceService(
    api_version=api_version,
    kind=constants.KFSERVING_KIND,
    metadata=client.V1ObjectMeta(
        name=inferenceServiceName, namespace=namespace, annotations=annotations
    ),
    spec=V1beta1InferenceServiceSpec(
        predictor=V1beta1PredictorSpec(
            min_replicas=1,
            sklearn=V1beta1SKLearnSpec(
                # Store your model with the name model.joblib and provide the file location
                # to the storage_uri variable
                storage_uri=inFerenceServiceStorageUri,
            ),
        )
    ),
)


## Create the InferenceService

In [ ]:
KFServing = KFServingClient()
KFServing.create(isvc)


In [ ]:
KFServing.get(inferenceServiceName, namespace=namespace, watch=True, timeout_seconds=120)

# Wait for the status READY True

## Test the InferenceService

In [ ]:
import json
import requests

import requests

response = requests.get(inputFile)
data = response.text

url = "http://" + inferenceServiceName + "." + namespace + ".svc.cluster.local/v1/models/" + inferenceServiceName + ":predict"

response = requests.post(url, json.dumps(json.loads(data)))

print("Input: ")
print(json.dumps(data))
print("\n")
print ("Output: ")
print(response.text)


## Delete the InferenceService

In [ ]:
KFServing.delete(inferenceServiceName, namespace=namespace)
